In [1]:
#https://dev.pokemontcg.io/dashboard
#https://github.com/PokemonTCG/pokemon-tcg-sdk-python

In [1]:
API_KEY = "1321d2cc-d928-4480-8d30-4e355eebc3aa"

In [4]:
!pip install pokemontcgsdk

You should consider upgrading via the 'c:\users\maxpr\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [5]:
from pokemontcgsdk import RestClient

RestClient.configure(API_KEY)

In [6]:
from pokemontcgsdk import Card
cards = Card.where(q='set.name:generations supertype:pokemon')

In [10]:
single_card = cards[0]

In [16]:
single_card.__dict__

{'abilities': None,
 'artist': 'Eske Yoshinob',
 'ancientTrait': None,
 'attacks': [Attack(name='Frog Hop', cost=['Grass', 'Colorless', 'Colorless'], convertedEnergyCost=3, damage='40+', text='Flip a coin. If heads, this attack does 40 more damage.'),
  Attack(name='Poison Impact', cost=['Grass', 'Grass', 'Colorless', 'Colorless'], convertedEnergyCost=4, damage='80', text="Your opponent's Active Pokémon is now Asleep and Poisoned.")],
 'cardmarket': Cardmarket(url='https://prices.pokemontcg.io/cardmarket/g1-1', updatedAt='2022/02/11', prices=CardmarketPrices(averageSellPrice=6.86, lowPrice=2.99, trendPrice=5.93, germanProLow=None, suggestedPrice=None, reverseHoloSell=None, reverseHoloLow=None, reverseHoloTrend=6.21, lowPriceExPlus=4.49, avg1=6.0, avg7=6.85, avg30=7.0, reverseHoloAvg1=6.99, reverseHoloAvg7=5.52, reverseHoloAvg30=5.03)),
 'convertedRetreatCost': 4,
 'evolvesFrom': None,
 'flavorText': None,
 'hp': '180',
 'id': 'g1-1',
 'images': CardImage(small='https://images.pokemontc

In [20]:
print(single_card.name)
print(single_card.nationalPokedexNumbers)
print(single_card.subtypes)
print(single_card.rarity)

Venusaur-EX
[3]
['Basic', 'EX']
Rare Holo EX


## For japanese

In [2]:
base_url = "https://jp.pokellector.com/sets"

In [3]:
from selenium import webdriver
import time
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd


In [4]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
# driver.implicitly_wait(10)
data = []

C:\Users\maxpr\AppData\Local\Temp\ipykernel_22504\2901737340.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )


In [8]:
from bs4 import BeautifulSoup

In [9]:
driver.get(base_url)

In [10]:
soup = BeautifulSoup(driver.page_source)

In [11]:
series_bs = soup.find_all('h1', class_="icon set")
blocks_bs = soup.find_all('div', class_="buttonlisting")

In [12]:
series = driver.find_elements_by_class_name('set')
blocks = driver.find_elements_by_class_name('buttonlisting')

C:\Users\maxpr\AppData\Local\Temp\ipykernel_22504\2139438567.py:1: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  series = driver.find_elements_by_class_name('set')
C:\Users\maxpr\AppData\Local\Temp\ipykernel_22504\2139438567.py:2: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  blocks = driver.find_elements_by_class_name('buttonlisting')


In [141]:
from clickhouse_driver import Client

In [142]:
db = Client("localhost")

In [149]:
extensions = db.query_dataframe(f"SELECT * FROM pokemon.extensions")

In [152]:
extensions['extensionUrl'].values

array(['https://jp.pokellector.com/sets/BWP-Black-White-Promos',
       'https://jp.pokellector.com/sets/KK-Blastoise-Kyurem-Combo-Deck',
       'https://jp.pokellector.com/sets/DS-Dragon-Selection',
       'https://jp.pokellector.com/sets/WAK-Exciting-Battle-for-Everyone',
       'https://jp.pokellector.com/sets/GBR-Garchomp-Half-Deck',
       'https://jp.pokellector.com/sets/SZD-Hydreigon-Half-Deck',
       'https://jp.pokellector.com/sets/KLD-Keldeo-Battle-Strength-Deck',
       'https://jp.pokellector.com/sets/MG-Mewtwo-Vs-Genesect-Genesect',
       'https://jp.pokellector.com/sets/MG-Mewtwo-Vs-Genesect-Mewtwo',
       'https://jp.pokellector.com/sets/SC-Shiny-Collection',
       'https://jp.pokellector.com/sets/PBG-Team-Plasma-Battle-Gift-Set',
       'https://jp.pokellector.com/sets/BW1-Black-Collection',
       'https://jp.pokellector.com/sets/BW6-Cold-Flare',
       'https://jp.pokellector.com/sets/BW4-Dark-Rush',
       'https://jp.pokellector.com/sets/BW5-Dragon-Blade',
     

In [18]:
from datetime import datetime, timedelta
from clickhouse_driver import Client
import pandas as pd


class DBHandler:

    def __init__(self, domain: str):
        self.client = Client(domain)

    def list_already_present_extensions(self):
        """Return the extensions code already present in the database.

        :rtype: List
        :return: List of String EXTENSION_CODE. 
        """
        return [val[0] for val in self.client.execute(f"SELECT DISTINCT extensionCode FROM pokemon.extensions")]

    def insert_extensions(self, dataframe: pd.DataFrame):
        """Insert into the database extensions that don't alreay exists.

        :param dataframe: Dataframe to insert containing extensions
        """
        # Declare 2 filters. Only take extensions not in DB, and that have released for more than 1 week.
        dataframe.to_csv('data.csv')
        already_present = self.list_already_present_extensions()
        latest_release_date = datetime.now() + timedelta(days=7)

        filtered_df = dataframe[(~dataframe["extensionReleaseDate"].isin(already_present)) & (dataframe["extensionReleaseDate"] < latest_release_date)]
        return self.client.execute(f"INSERT INTO pokemon.extensions VALUES", filtered_df.to_dict("records"), types_check=True)

    def get_oldest_extension(self):
        """Return the extensions code already present in the database.
        :rtype: List
        :return: List of String EXTENSION_CODE.
        """
        return [val[0] for val in self.client.execute(f"SELECT MAX(extensionReleaseDate) FROM pokemon.extensions")]


In [146]:
db.execute("ALTER TABLE pokemon.extensions DELETE where 1=1")

[]

In [24]:
db_handler = DBHandler("localhost")

In [3]:
db_handler.list_already_present_extensions()
df = pd.read_csv('../pokemon_data_scraper/data.csv')
df['extensionReleaseDate'] = df['extensionReleaseDate'].apply(lambda x: datetime.fromisoformat(x))

In [26]:
db_handler.get_oldest_extension()[0]

datetime.datetime(2022, 4, 8, 1, 5, tzinfo=<DstTzInfo 'Asia/Singapore' +08+8:00:00 STD>)

In [58]:
for i in range(2):
    for k in range(30):
        print(i, k)
        if k == 2:
            break

0 0
0 1
0 2
1 0
1 1
1 2


In [63]:
pd.DataFrame([],columns=["test", "nothing"])

,test,nothing


In [8]:
db.execute(f"INSERT INTO pokemon.extensions VALUES", df.to_dict("records"), types_check=True)

Error on localhost:9000 ping: Unexpected EOF while reading bytes
Connection was closed, reconnecting.


158

In [15]:
" ".join(test_1.split("\n")[-2:])

NameError: name 'test_1' is not defined

In [10]:
" ".join(test_2.split("\n")[-2:])

'Feb 25th 2022'

In [16]:
!pip install beautifulsoup4

You should consider upgrading via the 'C:\Users\maxpr\Documents\MyCode\pokemon-tcg-tracker\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [27]:
from bs4 import BeautifulSoup

In [131]:
type(bs_shit.find_all('div'))

bs4.element.ResultSet

In [28]:
from joblib import Parallel, delayed

In [123]:
import re

def test(extension):
    ext_code = extension.get_attribute('name')
    ext_url = extension.get_attribute('href')
    ext_name = extension.get_attribute('title')
    ext_image_url = extension.find_element_by_css_selector('img').get_attribute('src')
    print(ext_code)
    print(ext_url)
    print(ext_name)
    
    new_driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
    new_driver.get(ext_url)
    card_metadata = new_driver.find_element_by_class_name("setinfo").text
    #print(card_metadata)
    
    date = " ".join(card_metadata.split("\n")[-2:])
    
    new_driver.quit()
    #date_of_release = timestring.Date(date).date
    
    #card_number_str = " ".join(card_metadata.split("\n")[1:3]).split("+")
    #card_number = sum([int(re.sub(r'[^0-9]','',t)) for t in card_number_str])
    
    #print(f"Released in {date} with {card_number} cards")
    return [series[idx].text, ext_code, ext_url, ext_name, ext_image_url]

def test_bs(extension_bs):
    ext_code = extension_bs['name']
    ext_url = extension_bs['href']
    ext_name = extension_bs['title']
    ext_image_url = extension_bs.find('img')['src']
    print(ext_code)
    print(ext_url)
    print(ext_name)
    
    req = requests.get(base_url+ext_url.replace("/sets",""))
    bs_shit = BeautifulSoup(req.text)
    card_metadata = bs_shit.find('div', class_="setinfo").text
    card_number_str = " ".join(card_metadata.split("\n")[3:6]).split("+")
    card_number = sum([int(re.sub(r'[^0-9]','',t)) for t in card_number_str])
    #print(card_metadata)
    
    date = " ".join(card_metadata.split("\n")[-4:]).strip()
    print("nani")
    return [series[idx].text, ext_code, ext_url, ext_name, ext_image_url]

   

In [153]:

driver = new_driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
driver.get("https://jp.pokellector.com/sets/EBB-EX-Battle-Boost")
bs = BeautifulSoup(driver.page_source)
all_cards = bs.find_all('div' , class_="card")

C:\Users\maxpr\AppData\Local\Temp\ipykernel_22504\578358483.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = new_driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )


In [165]:
[card.find('a')['href'] for card in all_cards]

['/card/Shuckle-EX-Battle-Boost-EBB-1',
 '/card/Snivey-EX-Battle-Boost-EBB-2',
 '/card/Servine-EX-Battle-Boost-EBB-3',
 '/card/Serperior-EX-Battle-Boost-EBB-4',
 '/card/Sewaddle-EX-Battle-Boost-EBB-5',
 '/card/Swadloon-EX-Battle-Boost-EBB-6',
 '/card/Leavanny-EX-Battle-Boost-EBB-7',
 '/card/Virizion-EX-Battle-Boost-EBB-8',
 '/card/Charmander-EX-Battle-Boost-EBB-9',
 '/card/Charmeleon-EX-Battle-Boost-EBB-10',
 '/card/Charizard-EX-Battle-Boost-EBB-11',
 '/card/Vulpix-EX-Battle-Boost-EBB-12',
 '/card/Ninetales-EX-Battle-Boost-EBB-13',
 '/card/Moltres-EX-Battle-Boost-EBB-14',
 '/card/Victini-EX-Battle-Boost-EBB-15',
 '/card/Victini-EX-EX-Battle-Boost-EBB-16',
 '/card/Tepig-EX-Battle-Boost-EBB-17',
 '/card/Pignite-EX-Battle-Boost-EBB-18',
 '/card/Emboar-EX-Battle-Boost-EBB-19',
 '/card/Reshiram-EX-Battle-Boost-EBB-20',
 '/card/Reshiram-EX-EX-Battle-Boost-EBB-21',
 '/card/Magikarp-EX-Battle-Boost-EBB-22',
 '/card/Gyarados-EX-Battle-Boost-EBB-23',
 '/card/Articuno-EX-Battle-Boost-EBB-24',
 '/

In [192]:
card_r = requests.get(base_url.replace("/sets","")+'/card/Shuckle-EX-Battle-Boost-EBB-1')

In [162]:
base_url.replace("/sets","")+'/card/Shuckle-EX-Battle-Boost-EBB-1'

'https://jp.pokellector.com/card/Shuckle-EX-Battle-Boost-EBB-1'

In [194]:
bs = BeautifulSoup(card_r.text)

In [173]:
card_img

'https://den-cards.pokellector.com/129/Shuckle.EBB.1.png'

In [227]:
dict_card

{'JPN': 'ツボツボ', 'Rarity': 'Common', 'Set': 'EX Battle Boost', 'Card': '1/93'}

In [228]:
int('3o')

ValueError: invalid literal for int() with base 10: '3o'

In [246]:
card_name.contents[1]

'Pokemon Enterprise '

In [248]:

driver = new_driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
driver.get("https://jp.pokellector.com/sets/EBB-EX-Battle-Boost")
bs = BeautifulSoup(driver.page_source)
driver.quit()
all_cards = bs.find_all('div' , class_="card")
cards = []
# Define a default card number in case a series has no card number
card_number = 0
for index, card in enumerate(all_cards):
    try:
        card_page_request = requests.get(base_url.replace("/sets","")+card.find('a')['href'])
        card_bs = BeautifulSoup(card_page_request.text)
        card_info = card_bs.find('div', class_="infoblurb")
        card_img =  card_bs.find('div', class_="card").find('img')['src']
        card_name = card_bs.find('h1', class_="icon set")
        card_str_name = card_name.contents[1].strip()
        
        dict_card = {}
        for card_meta in card_info.find_all('div'):
            val = (card_meta.text.split(": "))
            dict_card[val[0]] = val[1]

        try:
            card_japanese_name = dict_card['JPN']
        except KeyError:
            card_japanese_name = ""
        
        try:
            card_rarity = dict_card['Rarity']
        except KeyError:
            card_rarity = "UNKNOWN"
        
        try:
            card_number = int(dict_card['Card'].split('/')[0])
        except (KeyError, ValueError):
            card_number = index
            LOGGER.error(f"Problem with card, name became {card_name} and number is {card_number}")

        cards.append([card_str_name, card_japanese_name, card_img, card_number, 'LOL', card_rarity])
    except Exception as e:
        print(base_url.replace("/sets","")+card.find('a')['href'])
        print(f"BAD STUFF for ulr {e}")
# return pd.DataFrame(cards, columns=[CardList.CARD_NAME, CardList.CARD_IMAGE_URL, CardList.CARD_NUMBER,
#                                     CardList.CARD_EXTENSION_CODE, CardList.CARD_RARITY])

C:\Users\maxpr\AppData\Local\Temp\ipykernel_22504\113171523.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = new_driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )


In [249]:
pd.DataFrame(cards, columns=["name", "jp_name","url","number", "code", "number"])

,name,jp_name,url,number,code,number
0,Shuckle,ツボツボ,https://den-cards.pokellector.com/129/Shuckle....,1,LOL,Common
1,Snivey,Snivey,https://den-cards.pokellector.com/129/Snivey.E...,2,LOL,Common
2,Servine,ジャノビー,https://den-cards.pokellector.com/129/Servine....,3,LOL,Common
3,Serperior,ジャローダ,https://den-cards.pokellector.com/129/Serperio...,4,LOL,Common
4,Sewaddle,クルミル,https://den-cards.pokellector.com/129/Sewaddle...,5,LOL,Common
...,...,...,...,...,...,...
94,Zekrom,ゼクロム,https://den-cards.pokellector.com/129/Zekrom.E...,95,LOL,Secret Rare
95,Imakuni,Imakuni,https://den-cards.pokellector.com/129/Imakuni....,96,LOL,Secret Rare
96,Pokemon Enterprise,ポケモン事業,https://den-cards.pokellector.com/129/Pokemon-...,97,LOL,Secret Rare
97,Pokemon Enterprise,ポケモン事業,https://den-cards.pokellector.com/129/Pokemon-...,98,LOL,Secret Rare


In [133]:
%%time
data_blocks_2 = []
for idx, block in enumerate(blocks_bs):
    print(series[idx].text)
    r = Parallel(n_jobs=10, backend="threading")(delayed(test_bs)(extension) for extension in block.find_all('a'))
    #print(ext.text)
    #ext_name = 
    for res in r:
        data_blocks_2.append(res)
    # for extension in block.find_elements_by_class_name('button'):
    #     
df_ext_2 = pd.DataFrame(data_blocks_2, columns =["serie", "ext_code", "ext_url", "ext_name", "ext_img_url"])

Sword & Shield Series
S10A
/sets/S10A-Dark-Phantasma
Dark Phantasma Set
S10P
/sets/S10P-Space-Juggler
Space Juggler Set
S10D
/sets/S10D-Time-Gazer
Time Gazer Set
S9A
/sets/S9A-Battle-Region
Battle Region Set
S9
/sets/S9-Star-Birth
Star Birth Set
SI
/sets/SI-Start-Deck-100
Start Deck 100 Set
S8B
/sets/S8B-VMAX-Climax
VMAX Climax Set
S8A
/sets/S8A-25th-Anniversary-Collection
25th Anniversary Collection Set
S8AP
/sets/S8AP-25th-Anniversary-Promo-Pack
25th Anniversary Promo Pack Set
S8
/sets/S8-Fusion-ARTS
Fusion ARTS Set
nani
S7R
/sets/S7R-Blue-Sky-Stream
Blue Sky Stream Set
nani
S7D
/sets/S7D-Towering-Perfection
Towering Perfection Set
nani
S6A
/sets/S6A-Eevee-Heroes
Eevee Heroes Set
nani
S6K
/sets/S6K-Jet-Black-Spirit
Jet Black Spirit Set
nani
S6H
/sets/S6H-Silver-Lance
Silver Lance Set
nani
S5A
/sets/S5A-Matchless-Fighter
Matchless Fighter Set
nani
S5R
/sets/S5R-Rapid-Strike-Master
Rapid Strike Master Set
nani
S5I
/sets/S5I-Single-Strike-Master
Single Strike Master Set
nani
S4A
/sets/S

In [127]:
%%time
data_blocks = []
for idx, block in enumerate(blocks_bs):
    print(series[idx].text)
    for extension in block.find_all('a'):
        data_blocks.append(test_bs(extension))

df_ext = pd.DataFrame(data_blocks, columns =["serie", "ext_code", "ext_url", "ext_name", "ext_img_url"])

Sword & Shield Series
S10A
/sets/S10A-Dark-Phantasma
Dark Phantasma Set
nani
S10P
/sets/S10P-Space-Juggler
Space Juggler Set
nani
S10D
/sets/S10D-Time-Gazer
Time Gazer Set
nani
S9A
/sets/S9A-Battle-Region
Battle Region Set
nani
S9
/sets/S9-Star-Birth
Star Birth Set
nani
SI
/sets/SI-Start-Deck-100
Start Deck 100 Set
nani
S8B
/sets/S8B-VMAX-Climax
VMAX Climax Set
nani
S8A
/sets/S8A-25th-Anniversary-Collection
25th Anniversary Collection Set
nani
S8AP
/sets/S8AP-25th-Anniversary-Promo-Pack
25th Anniversary Promo Pack Set
nani
S8
/sets/S8-Fusion-ARTS
Fusion ARTS Set
nani
S7R
/sets/S7R-Blue-Sky-Stream
Blue Sky Stream Set
nani
S7D
/sets/S7D-Towering-Perfection
Towering Perfection Set
nani
S6A
/sets/S6A-Eevee-Heroes
Eevee Heroes Set
nani
S6K
/sets/S6K-Jet-Black-Spirit
Jet Black Spirit Set
nani
S6H
/sets/S6H-Silver-Lance
Silver Lance Set
nani
S5A
/sets/S5A-Matchless-Fighter
Matchless Fighter Set
nani
S5R
/sets/S5R-Rapid-Strike-Master
Rapid Strike Master Set
nani
S5I
/sets/S5I-Single-Strike-Mas

In [134]:
df_ext == df_ext_2

,serie,ext_code,ext_url,ext_name,ext_img_url
0,True,True,True,True,True
1,True,True,True,True,True
2,True,True,True,True,True
3,True,True,True,True,True
4,True,True,True,True,True
...,...,...,...,...,...
154,True,True,True,True,True
155,True,True,True,True,True
156,True,True,True,True,True
157,True,True,True,True,True


In [10]:
%%time
data_blocks = []
for idx, block in enumerate(blocks):
    print(series[idx].text)
    r = Parallel(n_jobs=4, backend="loky")(delayed(test)(extension) for extension in block.find_elements_by_class_name('button'))
    #print(ext.text)
    #ext_name = 
    for res in r:
        data_blocks.append(res)
    # for extension in block.find_elements_by_class_name('button'):
    #     
df_ext = pd.DataFrame(data_blocks, columns =["serie", "ext_code", "ext_url", "ext_name", "ext_img_url"])

Sword & Shield Series


PicklingError: Could not pickle the task to send it to the workers.

In [ ]:
%%time
data_blocks_2 = []
for idx, block in enumerate(blocks):
    print(series[idx].text)
    for extension in block.find_elements_by_class_name('button'):
        data_blocks_2.append(test(extension))
df_ext = pd.DataFrame(data_blocks_2, columns =["serie", "ext_code", "ext_url", "ext_name", "ext_img_url"])

Sword & Shield Series
S10A
https://jp.pokellector.com/sets/S10A-Dark-Phantasma
Dark Phantasma Set


c:\Users\maxpr\Documents\MyCode\pokemon-tcg-tracker\venv\lib\site-packages\selenium\webdriver\remote\webelement.py:464: UserWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  warnings.warn("find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead")
c:\Users\maxpr\Documents\MyCode\pokemon-tcg-tracker\venv\lib\site-packages\selenium\webdriver\remote\webelement.py:484: UserWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  warnings.warn("find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead")
C:\Users\maxpr\AppData\Local\Temp\ipykernel_10944\1135022650.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  new_driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downl

S10P
https://jp.pokellector.com/sets/S10P-Space-Juggler
Space Juggler Set
S10D
https://jp.pokellector.com/sets/S10D-Time-Gazer
Time Gazer Set
S9A
https://jp.pokellector.com/sets/S9A-Battle-Region
Battle Region Set
S9
https://jp.pokellector.com/sets/S9-Star-Birth
Star Birth Set
SI
https://jp.pokellector.com/sets/SI-Start-Deck-100
Start Deck 100 Set
S8B
https://jp.pokellector.com/sets/S8B-VMAX-Climax
VMAX Climax Set
S8A
https://jp.pokellector.com/sets/S8A-25th-Anniversary-Collection
25th Anniversary Collection Set
S8AP
https://jp.pokellector.com/sets/S8AP-25th-Anniversary-Promo-Pack
25th Anniversary Promo Pack Set
S8
https://jp.pokellector.com/sets/S8-Fusion-ARTS
Fusion ARTS Set
S7R
https://jp.pokellector.com/sets/S7R-Blue-Sky-Stream
Blue Sky Stream Set
S7D
https://jp.pokellector.com/sets/S7D-Towering-Perfection
Towering Perfection Set
S6A
https://jp.pokellector.com/sets/S6A-Eevee-Heroes
Eevee Heroes Set
S6K
https://jp.pokellector.com/sets/S6K-Jet-Black-Spirit
Jet Black Spirit Set
S6H
ht

In [151]:
url_test = 'https://jp.pokellector.com/sets/S8-Fusion-ARTS'

In [152]:
driver.get(url_test)

In [153]:
card_metadata = driver.find_element_by_class_name("setinfo").text
print(card_metadata)

This set introduces new FUSION cards!
Cards
100
+29 Secret
Released
Sep 24th
2021


C:\Users\maxpr\AppData\Local\Temp/ipykernel_20160/3635570042.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  card_metadata = driver.find_element_by_class_name("setinfo").text


In [157]:
card_metadata = driver.find_element_by_class_name("setinfo").text
print(card_metadata)
card_metadata.split("Released")

This set introduces new FUSION cards!
Cards
100
+29 Secret
Released
Sep 24th
2021


C:\Users\maxpr\AppData\Local\Temp/ipykernel_20160/3946651833.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  card_metadata = driver.find_element_by_class_name("setinfo").text


['This set introduces new FUSION cards!\nCards\n100\n+29 Secret\n',
 '\nSep 24th\n2021']

In [155]:
all_cards = driver.find_element_by_class_name('cardlisting').find_elements_by_xpath('//div[@class="card "]')
cards = []
for card in all_cards:
    card_name = card.text
    card_img_ref = card.find_element_by_css_selector('a').find_element_by_css_selector('img').get_attribute('data-src')
    #print(card_name)
    #print(card_img_ref)
    cards.append([card_name, card_img_ref])
df_cards = pd.DataFrame(cards, columns = ['card_name','card_img_url'])

C:\Users\maxpr\AppData\Local\Temp/ipykernel_20160/4231425114.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  all_cards = driver.find_element_by_class_name('cardlisting').find_elements_by_xpath('//div[@class="card "]')
C:\Users\maxpr\AppData\Local\Programs\Python\Python38\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\maxpr\AppData\Local\Programs\Python\Python38\lib\site-packages\selenium\webdriver\remote\webelement.py:484: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


In [156]:
df_cards

,card_name,card_img_url
0,#1 - Caterpie,https://den-cards.pokellector.com/326/Caterpie...
1,#2 - Metapod,https://den-cards.pokellector.com/326/Metapod....
2,#3 - Butterfree,https://den-cards.pokellector.com/326/Butterfr...
3,#4 - Shroomish,https://den-cards.pokellector.com/326/Shroomis...
4,#5 - Breloom,https://den-cards.pokellector.com/326/Breloom....
...,...,...
124,#125 - Flaaffy,https://den-cards.pokellector.com/326/Flaaffy....
125,#126 - Power Tablet,https://den-cards.pokellector.com/326/Power-Ta...
126,#127 - Training Court,https://den-cards.pokellector.com/326/Training...
127,#128 - Grass Energy,https://den-cards.pokellector.com/326/Grass-En...


In [12]:
from tqdm.notebook import tqdm

def get_all_series(base_url: str) -> None:
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
    driver.implicitly_wait(10)
    driver.get(base_url)
    series = driver.find_elements_by_class_name('set')
    blocks = driver.find_elements_by_class_name('buttonlisting')
    data_blocks = []
    for idx, block in enumerate(tqdm(blocks[:3])):
    
        #print(ext.text)
        #ext_name = 
        for extension in block.find_elements_by_class_name('button'):
            ext_code = extension.get_attribute('name')
            ext_url = extension.get_attribute('href')
            ext_name = extension.get_attribute('title')
            ext_image_url = extension.find_element_by_css_selector('img').get_attribute('src')
            #print(ext_code)
            #print(ext_url)
            #print(ext_name)
            df_card_ext = one_extension_cards(ext_url)
            df_card_ext['series'] = series[idx].text
            df_card_ext['ext_code'] = ext_code
            df_card_ext['ext_name'] = ext_name
            #data_blocks.append([series[idx].text, ext_code, ext_url, ext_name, ext_image_url])
            df_card_ext.to_csv(ext_name+".csv")
    #df_ext = pd.DataFrame(data_blocks, columns =["serie", "ext_code", "ext_url", "ext_name", "ext_img_url"])


def one_extension_cards(url_extension: str) -> pd.DataFrame:
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
    driver.get(url_extension)
    card_metadata = driver.find_element_by_class_name("setinfo").text
    print(card_metadata)
    all_cards = driver.find_element_by_class_name('cardlisting').find_elements_by_xpath('//div[@class="card "]')
    cards = []
    for card in tqdm(all_cards):
        card_name = card.text
        card_img_ref = card.find_element_by_css_selector('a').find_element_by_css_selector('img').get_attribute('data-src')
        #print(card_name)
        #print(card_img_ref)
        cards.append([card_name, card_img_ref])
    df_cards = pd.DataFrame(cards, columns = ['card_name','card_img_url'])
    return df_cards

In [13]:
get_all_series(base_url)

C:\Users\maxpr\AppData\Local\Temp/ipykernel_23616/2609232765.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )
C:\Users\maxpr\AppData\Local\Temp/ipykernel_23616/2609232765.py:13: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  series = driver.find_elements_by_class_name('set')
C:\Users\maxpr\AppData\Local\Temp/ipykernel_23616/2609232765.py:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  blocks = driver.find_elements_by_class_name('buttonlisting')


  0%|          | 0/3 [00:00<?, ?it/s]

C:\Users\maxpr\AppData\Local\Programs\Python\Python38\lib\site-packages\selenium\webdriver\remote\webelement.py:464: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\maxpr\AppData\Local\Programs\Python\Python38\lib\site-packages\selenium\webdriver\remote\webelement.py:484: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
C:\Users\maxpr\AppData\Local\Temp/ipykernel_23616/2609232765.py:44: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=chrome_options,executable_path="C:\\Users\\maxpr\\Downloads\\chromedriver_win32\\chromedriver.exe" )


The Palkia half of the S10 set which introduces Hisuian Pokemon to the TCG
Cards
67
Released
Apr 8th
2022


C:\Users\maxpr\AppData\Local\Temp/ipykernel_23616/2609232765.py:46: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  card_metadata = driver.find_element_by_class_name("setinfo").text
C:\Users\maxpr\AppData\Local\Temp/ipykernel_23616/2609232765.py:48: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  all_cards = driver.find_element_by_class_name('cardlisting').find_elements_by_xpath('//div[@class="card "]')
C:\Users\maxpr\AppData\Local\Programs\Python\Python38\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


...

This set, featuring Arceus, introduced VSTAR Pokemon to the TCG!
Cards
100
+27 Secret
Released
Jan 14th
2022


  0%|          | 0/127 [00:00<?, ?it/s]

This "set" full of reprints contains 414 unique cards and is distributed via 100 different 60-card decks!
Cards
414
+8 Secret
Released
Dec 17th
2021


  0%|          | 0/430 [00:00<?, ?it/s]

KeyboardInterrupt: 